In [2]:
import math


class Cell:
    def __init__(self):
        self.devices = []


class Terrain:

    def __init__(self, size):
        self.size = size
        self.mobiles = [] #用来记录所有移动设备
        # self.self_hit, self.d2d_hit, self.bs_hit, self.sat_hit, self.miss = 0, 0, 0, 0, 0
        self.self_hit, self.d2d_hit, self.bs_hit,  self.miss = 0, 0, 0, 0
        self.cells = [[Cell() for _ in range(size)] for _ in range(size)] #用来记录所有的设备包括位置信息

    def clear_caches(self):
        # self.self_hit, self.d2d_hit, self.bs_hit, self.sat_hit, self.miss = 0, 0, 0, 0, 0
        self.self_hit, self.d2d_hit, self.bs_hit,  self.miss = 0, 0, 0, 0, 0
        # self.satellite.cache.clear()
        self.base_station.cache.clear()
        for m in self.mobiles:
            m.cache.clear()

    # 记录设备的位置信息
    def locate_device(self, device, x, y):
        self.cells[x][y].devices.append(device)

    # def add_satellite(self, satellite):
    #     self.satellite = satellite

    def add_base_station(self, base_station):
        self.base_station = base_station
        self.locate_device(self.base_station, self.base_station.x, self.base_station.y)

    def add_mobile(self, mobile):
        self.mobiles.append(mobile)
        self.locate_device(mobile, mobile.x, mobile.y)

    #计算两个设备之间的距离
    def distance_between(self, device1, device2):
        return math.sqrt((device1.x - device2.x) ** 2 + (device1.y - device2.y) ** 2)

    # 判断两个设备之间是否能够进行通信
    def can_communicate(self, device, other_device):
        return self.distance_between(device, other_device) < min(device.range, other_device.range)

    #当请求的特定内容已缓存在邻居节点中时，返回Ture
    def contains_in_neighbours(self, user, content):
        for m in self.mobiles:
            if self.can_communicate(user, m):
                if m.cache.contains(content):
                    m.cache.new_content(content)
                    return True

        return False

    def contains_in_base_station(self, user, content):
        if self.can_communicate(user, self.base_station):
            if self.base_station.cache.contains(content):
                self.base_station.cache.new_content(content)
                return True

        return False

    # def contains_in_satellite(self, content):
    #     if self.satellite.cache.contains(content):
    #         self.satellite.cache.new_content(content)
    #         return True
    #
    #     return False

    def content_request(self, user, content):
        # 本地缓存命中
        if user.cache.contains(content):
            self.self_hit += 1

        # D2D缓存命中
        elif self.contains_in_neighbours(user, content):
            self.d2d_hit += 1

        # 服务器缓存命中
        elif self.contains_in_base_station(user, content):
            self.bs_hit += 1

        # elif self.contains_in_satellite(content):
        #     self.sat_hit += 1

        else:
            if self.can_communicate(user, self.base_station):
                self.base_station.cache.new_content(content)
            else:
                self.satellite.cache.new_content(content)

            self.miss += 1

        # 缓存新内容
        user.cache.new_content(content)


In [7]:
A = [1,2,3,4,5,6]
B = [[A for _ in range(5)] for _ in range(5)]
print(len(B))
print(B)
print(B[2][4])

5
[[[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]], [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]], [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]], [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]], [[1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6]]]
[1, 2, 3, 4, 5, 6]
